# Create Combined All Data - Fixed Quarter Dates

1. credit_ratings_on_fixed_quarter_dates_with_earnings_call_date.csv  
        * Columns: rating, symbol, rating agency, rating_date, fixed_quarter_date, ...  
        * Size: 7981 × 16  
        * 587 unique companies
2. combined_calls.csv  
        * Columns: symbol, quarter', year, earnings_call_datetime, content, source, web, earnings_call_date  
        * Size: 31067 x 8  
        * 1646 unique companies  
3. combined_corrected_tabular_financial_statements_data.parquet  
        * Columns: date, symbol, reportedCurrency, period, filing_date, financial variables, Altman_Z ...  
        * Size: 20825 x 134  
        * 796 unique companies  

In [1]:
# Whether this is a sample/debug run or not
DEBUG = False

In [2]:
# Packages
import pandas as pd

In [3]:
# Load ~\Box\STAT 222 Capstone\Intermediate Data\Credit_Rating\credit_ratings_on_fixed_quarter_dates_with_earnings_call_date.csv
cr_and_dates = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Credit_Rating\credit_ratings_on_fixed_quarter_dates_with_earnings_call_date.csv')
cr_and_dates

,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,Source,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,next_rating_date_or_end_of_data,credit_rating_year,previous_fixed_quarter_date,source,days_since_call,days_since_rating
0,AAPL,2014-07-01 00:00:00.000000,2014-04-23,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,AAA,2014-04-24,2015-02-18,2014,2014-04-01 00:00:00.000000,web,69,35
1,AAPL,2014-10-01 00:00:00.000000,2014-07-22,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,AAA,2014-04-24,2015-02-18,2014,2014-07-01 00:00:00.000000,web,71,127
2,AAPL,2015-01-01 00:00:00.000000,2014-10-20,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,AAA,2014-04-24,2015-02-18,2014,2014-10-01 00:00:00.000000,web,73,219
3,AAPL,2015-04-01 00:00:00.000000,2015-01-27,AA,Standard & Poor's Ratings Services,2015-02-18,Supplementary,9,AA,2015-05-28,AA,2014-05-27,2015-05-28,2015,2015-01-01 00:00:00.000000,web,64,42
4,AAPL,2015-07-01 00:00:00.000000,2015-04-27,AA,Standard & Poor's Ratings Services,2015-06-02,Supplementary,9,AA,2015-08-25,AA,2015-05-28,2015-08-25,2015,2015-04-01 00:00:00.000000,web,65,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8008,ZTS,2015-10-01 00:00:00.000000,2015-08-04,BBB,Standard & Poor's Ratings Services,2015-01-30,Both,7,BBB,2015-11-03,BBB,2014-01-31,2015-11-03,2015,2015-07-01 00:00:00.000000,web,58,244
8009,ZTS,2016-01-01 00:00:00.000000,2015-11-03,BBB,Standard & Poor's Ratings Services,2015-11-03,Supplementary,7,BBB,2016-01-22,BBB,2015-01-30,2016-01-22,2015,2015-10-01 00:00:00.000000,web,59,59
8010,ZTS,2016-04-01 00:00:00.000000,2016-02-16,BBB,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,BBB,2015-11-03,2016-12-23,2016,2016-01-01 00:00:00.000000,web,45,70
8011,ZTS,2016-07-01 00:00:00.000000,2016-05-04,BBB,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,BBB,2015-11-03,2016-12-23,2016,2016-04-01 00:00:00.000000,web,58,161


In [4]:
# Convert earnings_call_date to date format
#cr_and_dates['earnings_call_date'] = pd.to_datetime(cr_and_dates['earnings_call_date']).dt.date

In [5]:
# drop source column (about to get it again via the next join)
cr_and_dates = cr_and_dates.drop(columns = ['source'])

In [6]:
# check for duplicates on ticker by fixed_quarter_date
print(cr_and_dates[cr_and_dates.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, Source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call, days_since_rating]
Index: []


In [7]:
# Load ~\Box\STAT 222 Capstone\Intermediate Data\Calls\calls.csv
if DEBUG:
    calls = pd.read_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\Calls\combined_calls.parquet', nrows = 10)
else:
    # calls = pd.read_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\Calls\combined_calls.parquet')
    calls = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Calls\combined_calls.csv')
calls

,symbol,quarter,year,earnings_call_date,content,earnings_call_datetime,source,web
0,BCE,4,2012,2013-02-07,operators from ourselves. Our business perform...,2013-02-07,web,True
1,BCE,3,2012,2012-11-01,"Operator: Good morning, ladies and gentlemen. ...",2012-11-01,web,True
2,BCE,2,2012,2012-08-08,"Operator: Good morning, ladies and gentlemen. ...",2012-08-08,web,True
3,BCE,4,2013,2014-02-06,"Operator: Good morning, ladies and gentlemen. ...",2014-02-06,web,True
4,BCE,3,2013,2013-11-07,"Operator: Good morning, ladies and gentlemen. ...",2013-11-07,web,True
...,...,...,...,...,...,...,...,...
18464,USM,2,2016,2016-08-05,"Operator\nGreetings, and welcome to the TDS an...",2016-08-05,web,True
18465,WFRD,3,2015,2015-10-22,"Operator\nGood morning. My name is Lori, and ...",2015-10-22,web,True
18466,WFRD,4,2015,2016-02-04,"Operator\nGood morning, my name is Laurie and ...",2016-02-04,web,True
18467,WFRD,1,2016,2016-05-05,"Operator\nGood morning. My name is Laurie, and...",2016-05-05,web,True


In [8]:
# check for duplicates on company by date
display(calls[calls.duplicated(subset=['symbol', 'earnings_call_date'], keep=False)])
calls.drop_duplicates(subset=['symbol', 'earnings_call_date'], inplace = True)
display(calls[calls.duplicated(subset=['symbol', 'earnings_call_date'], keep=False)])

,symbol,quarter,year,earnings_call_date,content,earnings_call_datetime,source,web
18355,CAG,1,2017,2016-09-29,"Operator: Good morning, and welcome to today's...",2016-09-29,web,True
18462,CAG,1,2017,2016-09-29,"Operator: Good morning, and welcome to today's...",2016-09-29,web,True


,symbol,quarter,year,earnings_call_date,content,earnings_call_datetime,source,web


# Earnings Call - Credit Score Merge

In [9]:
display(calls.columns)
display(cr_and_dates.columns)

Index(['symbol', 'quarter', 'year', 'earnings_call_date', 'content',
       'earnings_call_datetime', 'source', 'web'],
      dtype='object')

Index(['ticker', 'fixed_quarter_date', 'earnings_call_date', 'Rating',
       'Rating Agency Name', 'rating_date', 'Source', 'Rating Rank AAA is 10',
       'Next Rating', 'Next Rating Date', 'Previous Rating',
       'Previous Rating Date', 'next_rating_date_or_end_of_data',
       'credit_rating_year', 'previous_fixed_quarter_date', 'days_since_call',
       'days_since_rating'],
      dtype='object')

In [10]:
# convert earnings_call_date to date part
#cr_and_dates['earnings_call_date'] = pd.to_datetime(cr_and_dates['earnings_call_date']).dt.date
# drop earnings_call_datetime and web (identical as earnings_call_date and source)
calls.drop(["earnings_call_datetime", "web"], axis = 1, inplace=True)
# rename days_since_call and source 
cr_and_dates.rename(columns = {'days_since_call':'days_since_call_on_fixed_quarter', 'Source':'CR_source'}, inplace=True)
# rename symbol, source, and content
calls.rename(columns = {'symbol': 'ticker', "source": "Calls_source", "content": "transcript"}, inplace=True)

In [11]:
# print remaining columns
print(calls.columns)
print(calls.shape)
print(cr_and_dates.columns)
print(cr_and_dates.shape)
# Print type of columns
print(calls.dtypes)
print(cr_and_dates.dtypes)
# Print head of ticker, earnings_call_date in both datasets
print(calls[['ticker', 'earnings_call_date']].head())
print(cr_and_dates[['ticker', 'earnings_call_date']].head())
# Print all items with ticker == 'AAPL' in both datasets
print('aapl in calls')
print(calls[calls['ticker'] == 'AAPL'][['ticker', 'earnings_call_date']])
print('aapl in cr_and_dates')
print(cr_and_dates[cr_and_dates['ticker'] == 'AAPL'][['ticker', 'earnings_call_date']])

Index(['ticker', 'quarter', 'year', 'earnings_call_date', 'transcript',
       'Calls_source'],
      dtype='object')
(18468, 6)
Index(['ticker', 'fixed_quarter_date', 'earnings_call_date', 'Rating',
       'Rating Agency Name', 'rating_date', 'CR_source',
       'Rating Rank AAA is 10', 'Next Rating', 'Next Rating Date',
       'Previous Rating', 'Previous Rating Date',
       'next_rating_date_or_end_of_data', 'credit_rating_year',
       'previous_fixed_quarter_date', 'days_since_call_on_fixed_quarter',
       'days_since_rating'],
      dtype='object')
(8013, 17)
ticker                object
quarter                int64
year                   int64
earnings_call_date    object
transcript            object
Calls_source          object
dtype: object
ticker                              object
fixed_quarter_date                  object
earnings_call_date                  object
Rating                              object
Rating Agency Name                  object
rating_date            

In [12]:
# Inner join cr_and_dates (on columns ticker, earnings_call_date) with calls (on columns company, date)
merged = pd.merge(cr_and_dates, calls, on=['ticker', 'earnings_call_date'], how='inner')
print(len(merged))

# Convert earnings_call_date to date format
merged['earnings_call_date'] = pd.to_datetime(merged['earnings_call_date']).dt.date

# Range of date, year, credit_rating_year
for variable in ['earnings_call_date', 'year', 'credit_rating_year']:
    print(variable)
    print(merged[variable].min())
    print(merged[variable].max())
    print()

# Free up memory from calls
# del calls
# del call_with_dropout
# del cr_and_dates
merged.head()

8013
earnings_call_date
2010-05-03
2016-09-30

year
2010
2017

credit_rating_year
2010
2016



,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,Previous Rating Date,next_rating_date_or_end_of_data,credit_rating_year,previous_fixed_quarter_date,days_since_call_on_fixed_quarter,days_since_rating,quarter,year,transcript,Calls_source
0,AAPL,2014-07-01 00:00:00.000000,2014-04-23,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,2014-04-24,2015-02-18,2014,2014-04-01 00:00:00.000000,69,35,2,2014,"Operator: Good day, everyone, and welcome to t...",web
1,AAPL,2014-10-01 00:00:00.000000,2014-07-22,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,2014-04-24,2015-02-18,2014,2014-07-01 00:00:00.000000,71,127,3,2014,"Operator: Good day, everyone, and welcome to t...",web
2,AAPL,2015-01-01 00:00:00.000000,2014-10-20,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,2014-04-24,2015-02-18,2014,2014-10-01 00:00:00.000000,73,219,4,2014,"Operator: Good day, everyone, and welcome to t...",web
3,AAPL,2015-04-01 00:00:00.000000,2015-01-27,AA,Standard & Poor's Ratings Services,2015-02-18,Supplementary,9,AA,2015-05-28,...,2014-05-27,2015-05-28,2015,2015-01-01 00:00:00.000000,64,42,1,2015,Operator: Good day ladies and gentlemen and we...,web
4,AAPL,2015-07-01 00:00:00.000000,2015-04-27,AA,Standard & Poor's Ratings Services,2015-06-02,Supplementary,9,AA,2015-08-25,...,2015-05-28,2015-08-25,2015,2015-04-01 00:00:00.000000,65,29,2,2015,"Operator: Good day everyone, and welcome to th...",web


In [13]:
# Keep only the date 
merged["fixed_quarter_date"] = pd.to_datetime(merged["fixed_quarter_date"]).dt.date
merged["previous_fixed_quarter_date"] = pd.to_datetime(merged["previous_fixed_quarter_date"]).dt.date
merged["rating_date"] = pd.to_datetime(merged["rating_date"]).dt.date

In [14]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, CR_source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call_on_fixed_quarter, days_since_rating, quarter, year, transcript, Calls_source]
Index: []

[0 rows x 21 columns]


In [15]:
# check values of source
print(merged['Calls_source'].value_counts())

Calls_source
web    8013
Name: count, dtype: int64


In [16]:
print(merged.shape)
print(merged.columns)

(8013, 21)
Index(['ticker', 'fixed_quarter_date', 'earnings_call_date', 'Rating',
       'Rating Agency Name', 'rating_date', 'CR_source',
       'Rating Rank AAA is 10', 'Next Rating', 'Next Rating Date',
       'Previous Rating', 'Previous Rating Date',
       'next_rating_date_or_end_of_data', 'credit_rating_year',
       'previous_fixed_quarter_date', 'days_since_call_on_fixed_quarter',
       'days_since_rating', 'quarter', 'year', 'transcript', 'Calls_source'],
      dtype='object')


In [17]:
print("number of fix_quarter_date = rating_date:", sum(merged.fixed_quarter_date == merged.rating_date))
print("number of fix_quarter_date after rating_date:", sum(merged.fixed_quarter_date < merged.rating_date))
print("number of earnings_call_date = rating_date:", sum(merged.earnings_call_date == merged.rating_date))
print("number of earnings_call_date after rating_date:", sum(merged.earnings_call_date < merged.rating_date))

number of fix_quarter_date = rating_date: 26
number of fix_quarter_date after rating_date: 0
number of earnings_call_date = rating_date: 47
number of earnings_call_date after rating_date: 1619


In [18]:
merged.days_since_call_on_fixed_quarter.describe()

count    8013.000000
mean       56.980032
std        14.821651
min         0.000000
25%        54.000000
50%        61.000000
75%        66.000000
max        91.000000
Name: days_since_call_on_fixed_quarter, dtype: float64

In [19]:
# Get columns as for_year and for_quarter
merged.rename(columns={"year": "for_year", "quarter": "for_quarter"}, inplace=True)

In [20]:
merged[merged.for_year != merged.credit_rating_year].head(10)

,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,Previous Rating Date,next_rating_date_or_end_of_data,credit_rating_year,previous_fixed_quarter_date,days_since_call_on_fixed_quarter,days_since_rating,for_quarter,for_year,transcript,Calls_source
7,AAPL,2016-04-01,2016-01-26,AA,Standard & Poor's Ratings Services,2015-08-25,Supplementary,9,AA,2016-05-20,...,2015-06-02,2016-05-20,2015,2016-01-01,66,220,1,2016,"Operator: Good day, ladies and gentlemen, and ...",web
13,ABB,2013-07-01,2013-04-24,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2013-04-01,68,410,1,2013,"Operator: Ladies and gentlemen, good morning o...",web
14,ABB,2013-10-01,2013-07-25,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2013-07-01,68,502,2,2013,"Operator: Ladies and gentlemen, good morning o...",web
15,ABB,2014-01-01,2013-10-24,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2013-10-01,69,594,3,2013,"Operator: Ladies and gentlemen, good morning o...",web
16,ABB,2014-04-01,2014-02-13,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2014-01-01,47,684,4,2013,operator and the maintenance leader wants he c...,web
17,ABB,2014-07-01,2014-04-29,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2014-04-01,63,775,1,2014,"Operator: Ladies and gentlemen, good morning, ...",web
18,ABB,2014-10-01,2014-07-23,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2014-07-01,70,867,2,2014,"Operator: Ladies and gentlemen, good morning o...",web
19,ABB,2015-01-01,2014-10-22,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2014-10-01,71,959,3,2014,"Operator: Ladies and gentlemen, good morning o...",web
20,ABB,2015-04-01,2015-02-05,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2015-01-01,55,1049,4,2014,Operator: First question is from Mr. Ben Uglow...,web
21,ABB,2015-07-01,2015-04-29,A,Standard & Poor's Ratings Services,2012-05-17,Supplementary,8,NaN,NaN,...,2012-02-01,2016-12-31,2012,2015-04-01,63,1140,1,2015,"Operator: Ladies and gentlemen, good morning o...",web


In [21]:
merged.columns

Index(['ticker', 'fixed_quarter_date', 'earnings_call_date', 'Rating',
       'Rating Agency Name', 'rating_date', 'CR_source',
       'Rating Rank AAA is 10', 'Next Rating', 'Next Rating Date',
       'Previous Rating', 'Previous Rating Date',
       'next_rating_date_or_end_of_data', 'credit_rating_year',
       'previous_fixed_quarter_date', 'days_since_call_on_fixed_quarter',
       'days_since_rating', 'for_quarter', 'for_year', 'transcript',
       'Calls_source'],
      dtype='object')

### Merge Financial Statements Data to CR-Calls data

In [22]:
# Load tabular financial statement data
df = pd.read_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\Tabular_Fin\combined_corrected_tabular_financial_statements_data.parquet')

# Perform merge
merged = pd.merge(merged, df, on=['ticker', 'for_year', 'for_quarter'], how='inner')    
display(merged.head())

,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,EBIT,common_plus_preferred_stock,workingCapital,Ratio_A,Ratio_B,Ratio_C,Ratio_D,Ratio_E,Altman_Z,filingDate
0,AAPL,2014-07-01,2014-04-23,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,13593000.0,21496000.0,27333000.0,0.065989,0.221594,5.471793,0.132692,0.480288,4.551851,2014-04-24
1,AAPL,2014-10-01,2014-07-22,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,10282000.0,22139000.0,21744000.0,0.046207,0.168219,5.445584,0.097717,0.443623,4.324703,2014-07-23
2,AAPL,2015-01-01,2014-10-20,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,11165000.0,23313000.0,5083000.0,0.048158,0.181691,8.059440,0.021925,0.375916,5.727053,2014-10-27
3,AAPL,2015-04-01,2015-01-27,AA,Standard & Poor's Ratings Services,2015-02-18,Supplementary,9,AA,2015-05-28,...,24246000.0,24187000.0,9792000.0,0.079295,0.284844,4.807177,0.037389,0.371059,3.992323,2015-01-28
4,AAPL,2015-07-01,2015-04-27,AA,Standard & Poor's Ratings Services,2015-06-02,Supplementary,9,AA,2015-08-25,...,18278000.0,25376000.0,9162000.0,0.069979,0.222095,8.059440,0.035077,0.386379,5.869492,2015-04-28


In [23]:
# Convert filingDate to date format
merged['filingDate'] = pd.to_datetime(merged['filingDate']).dt.date

In [24]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, CR_source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call_on_fixed_quarter, days_since_rating, for_quarter, for_year, transcript, Calls_source, reportedCurrency, acceptedDate_balance_sheet, cashAndCashEquivalents, shortTermInvestments, cashAndShortTermInvestments, netReceivables, inventory_balance_sheet, otherCurrentAssets, totalCurrentAssets, propertyPlantEquipmentNet, goodwill, intangibleAssets, goodwillAndIntangibleAssets, longTermInvestments, taxAssets, otherNonCurrentAssets, totalNonCurrentAssets, otherAssets, totalAssets, accountPayables, shortTermDebt, taxPayables, deferredRevenue, otherCurrentLiabilities, totalCurrentLiabilities, longTermDebt, deferredRevenueNonCurrent, deferredTaxLiabilitiesNonCurrent, otherNonCurrentLiabi

### Drops based on date disagreements

In [25]:
# Drop if filingDate is not in the fixed quarter
print('num cases where filingDate not in fixed quarter', len(merged[(merged['filingDate'] < merged['previous_fixed_quarter_date']) | (merged['filingDate'] > merged['fixed_quarter_date'])]))
merged = merged[(merged['filingDate'] >= merged['previous_fixed_quarter_date']) & (merged['filingDate'] <= merged['fixed_quarter_date'])]
# Drop if filingDate more than 45 days after earnings_call_date
print('num cases where filingDate more than 45 days after earnings_call_date', len(merged[(merged['filingDate'] - merged['earnings_call_date']) > pd.Timedelta('45 days 00:00:00')]))
merged = merged[(merged['filingDate'] - merged['earnings_call_date']) <= pd.Timedelta('45 days 00:00:00')]

num cases where filingDate not in fixed quarter 658
num cases where filingDate more than 45 days after earnings_call_date 3


### New Columns: Rating Changes

In [26]:
# Variable for rating on previous fixed_quarter_date
# Sort by ticker, fixed_quarter_date
merged = merged.sort_values(by=['ticker', 'fixed_quarter_date'])
# Shift the rating by 1
merged['rating_on_previous_fixed_quarter_date'] = merged.groupby('ticker')['Rating'].shift(1)
merged[['ticker', 'fixed_quarter_date', 'Rating', 'rating_on_previous_fixed_quarter_date']].head(10)

,ticker,fixed_quarter_date,Rating,rating_on_previous_fixed_quarter_date
0,AAPL,2014-07-01,AA,NaN
1,AAPL,2014-10-01,AA,AA
2,AAPL,2015-01-01,AA,AA
3,AAPL,2015-04-01,AA,AA
4,AAPL,2015-07-01,AA,AA
5,AAPL,2015-10-01,AA,AA
6,AAPL,2016-01-01,AA,AA
7,AAPL,2016-04-01,AA,AA
8,AAPL,2016-07-01,AA,AA
9,AAPL,2016-10-01,AA,AA


In [27]:
# Rating change variables
# Get numeric values for Rating and rating_on_next_earnings_call_date
# Encode Ratings as ordinal
rating_order = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'CC', 'C', 'D']
# Reverse
rating_order = rating_order[::-1]
# Create dictionary mapping to number values
rating_dict = {rating: i for i, rating in enumerate(rating_order, 1)}
# Create new column "Rating Rank AAA is 10"
merged['Rating Rank AAA is 10'] = merged['Rating'].map(rating_dict)
# Flag for investment grade - Rating Rank >= 7.0
merged['Investment_Grade'] = merged['Rating Rank AAA is 10'] >= 7.0
# Create new column "rating_on_previous_fixed_quarter_date AAA is 10"
merged['rating_on_previous_fixed_quarter_date AAA is 10'] = merged['rating_on_previous_fixed_quarter_date'].map(rating_dict)

# Cross tab Rating and 'Rating Rank AAA is 10'
display(pd.crosstab(merged['Rating'], merged['Rating Rank AAA is 10']))
# Cross tab rating_on_previous_fixed_quarter_date and 'rating_on_previous_fixed_quarter_date AAA is 10'
display(pd.crosstab(merged['rating_on_previous_fixed_quarter_date'], merged['rating_on_previous_fixed_quarter_date AAA is 10']))

Rating Rank AAA is 10,1,2,3,4,5,6,7,8,9,10
Rating,,,,,,,,,,
A,0,0,0,0,0,0,0,1209,0,0
AA,0,0,0,0,0,0,0,0,255,0
AAA,0,0,0,0,0,0,0,0,0,128
B,0,0,0,0,1061,0,0,0,0,0
BB,0,0,0,0,0,1758,0,0,0,0
BBB,0,0,0,0,0,0,2074,0,0,0
C,0,16,0,0,0,0,0,0,0,0
CC,0,0,14,0,0,0,0,0,0,0
CCC,0,0,0,165,0,0,0,0,0,0


rating_on_previous_fixed_quarter_date AAA is 10,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
rating_on_previous_fixed_quarter_date,,,,,,,,,,
A,0,0,0,0,0,0,0,1129,0,0
AA,0,0,0,0,0,0,0,0,230,0
AAA,0,0,0,0,0,0,0,0,0,120
B,0,0,0,0,971,0,0,0,0,0
BB,0,0,0,0,0,1616,0,0,0,0
BBB,0,0,0,0,0,0,1930,0,0,0
C,0,15,0,0,0,0,0,0,0,0
CC,0,0,13,0,0,0,0,0,0,0
CCC,0,0,0,154,0,0,0,0,0,0


In [28]:
# Variable for change before next call
def change(row):
    if row['Rating Rank AAA is 10'] < row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Downgrade Since Last Fixed Quarter Date'
    elif row['Rating Rank AAA is 10'] > row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Upgrade Since Last Fixed Quarter Date'
    elif row['Rating Rank AAA is 10'] == row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Same As Last Fixed Quarter Date'
    else:
        return None

# Apply function to create new column
merged['Change Direction Since Last Fixed Quarter Date'] = merged.apply(change, axis = 1)

# Also numeric version 'Change Since Last Fixed Quarter Date'
merged['Change Since Last Fixed Quarter Date'] = merged['Rating Rank AAA is 10'] - merged['rating_on_previous_fixed_quarter_date AAA is 10']

# Cross tab 'Change Direction Since Last Fixed Quarter Date' and 'Change Since Last Fixed Quarter Date'
display(pd.crosstab(merged['Change Direction Since Last Fixed Quarter Date'], merged['Change Since Last Fixed Quarter Date']))

Change Since Last Fixed Quarter Date,-2.0,-1.0,0.0,1.0,2.0
Change Direction Since Last Fixed Quarter Date,,,,,
Downgrade Since Last Fixed Quarter Date,16,127,0,0,0
Same As Last Fixed Quarter Date,0,0,5860,0,0
Upgrade Since Last Fixed Quarter Date,0,0,0,165,18


### Sector and Other Textual Company Information

From Kaggle: https://www.kaggle.com/datasets/aramacus/usa-public-companies

Supplemented with manually looked up sector data for missing sectors

In [29]:
# Load CSV 'C:\Users\ijyli\Box\STAT 222 Capstone\Intermediate Data\Sectors\combined_sector_data.csv'
combined_sector_data = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Sectors\combined_sector_data.csv')
combined_sector_data.head()

,ticker,Sector
0,CTVA,Materials
1,ALCO,Consumer Staples
2,LMNR,Consumer Staples
3,SANW,Consumer Staples
4,TRC,Real Estate


In [30]:
combined_sector_data.shape

(3389, 2)

In [31]:
# Left join us_stocks with merged
merged = pd.merge(merged, combined_sector_data, on='ticker', how='left')
merged

,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,CR_source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,Ratio_D,Ratio_E,Altman_Z,filingDate,rating_on_previous_fixed_quarter_date,Investment_Grade,rating_on_previous_fixed_quarter_date AAA is 10,Change Direction Since Last Fixed Quarter Date,Change Since Last Fixed Quarter Date,Sector
0,AAPL,2014-07-01,2014-04-23,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,0.132692,0.480288,4.551851,2014-04-24,NaN,True,NaN,None,NaN,Information Technology
1,AAPL,2014-10-01,2014-07-22,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,0.097717,0.443623,4.324703,2014-07-23,AA,True,9.0,Same As Last Fixed Quarter Date,0.0,Information Technology
2,AAPL,2015-01-01,2014-10-20,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,0.021925,0.375916,5.727053,2014-10-27,AA,True,9.0,Same As Last Fixed Quarter Date,0.0,Information Technology
3,AAPL,2015-04-01,2015-01-27,AA,Standard & Poor's Ratings Services,2015-02-18,Supplementary,9,AA,2015-05-28,...,0.037389,0.371059,3.992323,2015-01-28,AA,True,9.0,Same As Last Fixed Quarter Date,0.0,Information Technology
4,AAPL,2015-07-01,2015-04-27,AA,Standard & Poor's Ratings Services,2015-06-02,Supplementary,9,AA,2015-08-25,...,0.035077,0.386379,5.869492,2015-04-28,AA,True,9.0,Same As Last Fixed Quarter Date,0.0,Information Technology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6684,ZTS,2015-10-01,2015-08-04,BBB,Standard & Poor's Ratings Services,2015-01-30,Both,7,BBB,2015-11-03,...,0.264323,0.120491,3.500267,2015-08-06,BBB,True,7.0,Same As Last Fixed Quarter Date,0.0,Health Care
6685,ZTS,2016-01-01,2015-11-03,BBB,Standard & Poor's Ratings Services,2015-11-03,Supplementary,7,BBB,2016-01-22,...,0.268023,0.141041,3.198497,2015-11-05,BBB,True,7.0,Same As Last Fixed Quarter Date,0.0,Health Care
6686,ZTS,2016-04-01,2016-02-16,BBB,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,...,0.258941,0.110704,2.782747,2016-02-24,BBB,True,7.0,Same As Last Fixed Quarter Date,0.0,Health Care
6687,ZTS,2016-07-01,2016-05-04,BBB,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,...,0.287669,0.136321,3.000131,2016-05-06,BBB,True,7.0,Same As Last Fixed Quarter Date,0.0,Health Care


In [32]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, CR_source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call_on_fixed_quarter, days_since_rating, for_quarter, for_year, transcript, Calls_source, reportedCurrency, acceptedDate_balance_sheet, cashAndCashEquivalents, shortTermInvestments, cashAndShortTermInvestments, netReceivables, inventory_balance_sheet, otherCurrentAssets, totalCurrentAssets, propertyPlantEquipmentNet, goodwill, intangibleAssets, goodwillAndIntangibleAssets, longTermInvestments, taxAssets, otherNonCurrentAssets, totalNonCurrentAssets, otherAssets, totalAssets, accountPayables, shortTermDebt, taxPayables, deferredRevenue, otherCurrentLiabilities, totalCurrentLiabilities, longTermDebt, deferredRevenueNonCurrent, deferredTaxLiabilitiesNonCurrent, otherNonCurrentLiabi

In [33]:
merged.Sector.isnull().sum()

0

## Exporting Data

In [34]:
# Clean up variables

# print column names
for col in merged.columns:
    print(col)

ticker
fixed_quarter_date
earnings_call_date
Rating
Rating Agency Name
rating_date
CR_source
Rating Rank AAA is 10
Next Rating
Next Rating Date
Previous Rating
Previous Rating Date
next_rating_date_or_end_of_data
credit_rating_year
previous_fixed_quarter_date
days_since_call_on_fixed_quarter
days_since_rating
for_quarter
for_year
transcript
Calls_source
reportedCurrency
acceptedDate_balance_sheet
cashAndCashEquivalents
shortTermInvestments
cashAndShortTermInvestments
netReceivables
inventory_balance_sheet
otherCurrentAssets
totalCurrentAssets
propertyPlantEquipmentNet
goodwill
intangibleAssets
goodwillAndIntangibleAssets
longTermInvestments
taxAssets
otherNonCurrentAssets
totalNonCurrentAssets
otherAssets
totalAssets
accountPayables
shortTermDebt
taxPayables
deferredRevenue
otherCurrentLiabilities
totalCurrentLiabilities
longTermDebt
deferredRevenueNonCurrent
deferredTaxLiabilitiesNonCurrent
otherNonCurrentLiabilities
totalNonCurrentLiabilities
otherLiabilities
capitalLeaseObligations


In [35]:
# delete columns Rating Agency Name, CR_Source, Calls_Source
merged.drop(columns=['Rating Agency Name', 'CR_source', 'Calls_source'], inplace=True)

In [40]:
# remove some drop_out
mask = merged.ticker.isin(["DRI", "MU", "XIN", "DEO"])

merged = merged[~mask]

In [41]:
# Save a parquet file
merged.to_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates.parquet', index=False)

In [42]:
# Sample 100 rows to create all_data_sample.csv
if not DEBUG:
    all_data_sample = merged.sample(100)
    all_data_sample.to_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates_sample.csv', index=False)
    # also save to xlsx
    all_data_sample.to_excel(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates_sample.xlsx', index=False)

## Fragmented Version to Store on GitHub

In [43]:
# Split dataset into pieces
num_pieces = 8
total_len_pieces = 0
# Delete previous pieces, all contents of '../../../Data/All_Data/All_Data_Fixed_Quarter_Dates'
import os
import shutil
folder = r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))
# Save pieces
if not DEBUG:
    for i in range(num_pieces):
        # start index for piece rows
        start_index = i * len(merged) // num_pieces
        # end index for piece rows
        end_index = (i + 1) * len(merged) // num_pieces
        # get piece
        piece = merged[start_index:end_index]
        piece.to_parquet(r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates/all_data_fixed_quarter_dates_piece_' + str(i) + '.parquet', index=False)
        display(piece.head(3))
        print(len(piece))
        total_len_pieces += len(piece)
    # save 100 row sample
    merged.sample(100).to_parquet(r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates_Sample/all_data_fixed_quarter_dates_sample.parquet', index=False)


,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,...,Ratio_D,Ratio_E,Altman_Z,filingDate,rating_on_previous_fixed_quarter_date,Investment_Grade,rating_on_previous_fixed_quarter_date AAA is 10,Change Direction Since Last Fixed Quarter Date,Change Since Last Fixed Quarter Date,Sector
0,AAPL,2014-07-01,2014-04-23,AA,2014-05-27,9,AA,2015-02-18,AAA,2014-04-24,...,0.132692,0.480288,4.551851,2014-04-24,NaN,True,NaN,None,NaN,Information Technology
1,AAPL,2014-10-01,2014-07-22,AA,2014-05-27,9,AA,2015-02-18,AAA,2014-04-24,...,0.097717,0.443623,4.324703,2014-07-23,AA,True,9.0,Same As Last Fixed Quarter Date,0.0,Information Technology
2,AAPL,2015-01-01,2014-10-20,AA,2014-05-27,9,AA,2015-02-18,AAA,2014-04-24,...,0.021925,0.375916,5.727053,2014-10-27,AA,True,9.0,Same As Last Fixed Quarter Date,0.0,Information Technology


835


,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,...,Ratio_D,Ratio_E,Altman_Z,filingDate,rating_on_previous_fixed_quarter_date,Investment_Grade,rating_on_previous_fixed_quarter_date AAA is 10,Change Direction Since Last Fixed Quarter Date,Change Since Last Fixed Quarter Date,Sector
835,BZH,2014-10-01,2014-07-31,CCC,2014-07-31,4,CCC,2015-07-27,CCC,2014-06-13,...,0.570362,-0.319159,0.662515,2014-07-31,CCC,False,4.0,Same As Last Fixed Quarter Date,0.0,Consumer Discretionary
836,BZH,2015-01-01,2014-11-12,CCC,2014-07-31,4,CCC,2015-07-27,CCC,2014-06-13,...,0.570362,-0.276474,0.772045,2014-11-13,CCC,False,4.0,Same As Last Fixed Quarter Date,0.0,Consumer Discretionary
837,BZH,2015-04-01,2015-01-30,CCC,2014-07-31,4,CCC,2015-07-27,CCC,2014-06-13,...,0.570362,-0.298382,0.560885,2015-01-30,CCC,False,4.0,Same As Last Fixed Quarter Date,0.0,Consumer Discretionary


835


,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,...,Ratio_D,Ratio_E,Altman_Z,filingDate,rating_on_previous_fixed_quarter_date,Investment_Grade,rating_on_previous_fixed_quarter_date AAA is 10,Change Direction Since Last Fixed Quarter Date,Change Since Last Fixed Quarter Date,Sector
1676,EAT,2016-04-01,2016-01-20,BBB,2014-05-28,7,BBB,2016-08-11,BBB,2013-08-02,...,-0.123472,1.010775,2.822223,2016-02-01,BBB,True,7.0,Same As Last Fixed Quarter Date,0.0,Consumer Discretionary
1677,EAT,2016-07-01,2016-04-19,BBB,2014-05-28,7,BBB,2016-08-11,BBB,2013-08-02,...,-0.127414,1.010775,2.890326,2016-05-02,BBB,True,7.0,Same As Last Fixed Quarter Date,0.0,Consumer Discretionary
1678,EAT,2016-10-01,2016-08-11,BB,2016-09-13,6,NaN,NaN,BBB,2016-08-11,...,-0.127414,1.010775,3.003020,2016-08-29,BBB,False,7.0,Downgrade Since Last Fixed Quarter Date,-1.0,Consumer Discretionary


...

835


,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,...,Ratio_D,Ratio_E,Altman_Z,filingDate,rating_on_previous_fixed_quarter_date,Investment_Grade,rating_on_previous_fixed_quarter_date AAA is 10,Change Direction Since Last Fixed Quarter Date,Change Since Last Fixed Quarter Date,Sector
5017,PPG,2014-01-01,2013-10-17,A,2013-06-20,8,A,2014-03-03,BBB,2012-07-31,...,0.284493,0.952774,3.912420,2013-10-28,A,True,8.0,Same As Last Fixed Quarter Date,0.0,Materials
5018,PPG,2014-04-01,2014-01-16,A,2014-03-03,8,A,2015-03-11,A,2013-06-20,...,0.194099,0.804198,3.785986,2014-02-20,A,True,8.0,Same As Last Fixed Quarter Date,0.0,Materials
5019,PPG,2014-07-01,2014-04-17,A,2014-03-03,8,A,2015-03-11,A,2013-06-20,...,0.252498,0.814045,3.760203,2014-04-24,A,True,8.0,Same As Last Fixed Quarter Date,0.0,Materials


835


,ticker,fixed_quarter_date,earnings_call_date,Rating,rating_date,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,...,Ratio_D,Ratio_E,Altman_Z,filingDate,rating_on_previous_fixed_quarter_date,Investment_Grade,rating_on_previous_fixed_quarter_date AAA is 10,Change Direction Since Last Fixed Quarter Date,Change Since Last Fixed Quarter Date,Sector
5852,TEN,2013-04-01,2013-01-31,B,2013-03-28,5,BB,2014-04-30,B,2012-03-08,...,0.131652,-0.583149,NaN,2013-02-28,B,False,5.0,Same As Last Fixed Quarter Date,0.0,Consumer Discretionary
5853,TEN,2013-07-01,2013-04-29,B,2013-03-28,5,BB,2014-04-30,B,2012-03-08,...,0.178700,-0.528623,NaN,2013-05-07,B,False,5.0,Same As Last Fixed Quarter Date,0.0,Consumer Discretionary
5854,TEN,2013-10-01,2013-07-29,B,2013-03-28,5,BB,2014-04-30,B,2012-03-08,...,0.165778,-0.504443,NaN,2013-08-06,B,False,5.0,Same As Last Fixed Quarter Date,0.0,Consumer Discretionary


836


In [44]:
# check total piece length and length of merged
print(total_len_pieces)
print(len(merged))

6681
6681
